In [17]:
#Importing necessary packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from os import listdir
import cv2
from matplotlib import image
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation,MaxPooling2D,Dropout
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from math import sqrt
from matplotlib import image
from keras.layers import GaussianNoise
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings("ignore")

In [7]:
%cd C:\Users\praga\Downloads\ChromDownloads\

C:\Users\praga\Downloads\ChromDownloads


In [4]:
#Loading Good Images, converting to grayscale and saving separately and Resizing to (300,200) and saving separatley.
for filename in listdir('maskImages/good'):   
    # load the image
    image = Image.open('maskImages/good/' + filename)
    gs_image = image.convert(mode='L') #Grayscale
    image=image.resize((300,200)) #Resize
    # save in jpeg format
    gs_image.save('maskImages/goodGrayscale/gray_' + filename)  #Save grayscale images
    image.save('maskImages/goodResized/resize_' + filename)    #Save resized images

In [13]:
#Loading Bad Images, converting to grayscale and saving separately and Resizing to (300,200) and saving separatley.
for filename in listdir('maskImages/bad'):
    # load the image
    image = Image.open('maskImages/bad/' + filename)
    gs_image = image.convert(mode='L')   #Grayscale
    image=image.resize((300,200))    #Resize
    # save in jpeg format
    gs_image.save('maskImages/badGrayscale/gray_' + filename)    #Save grayscale images
    image.save('maskImages/badResized/resize_' + filename)       #Save resized images

In [ ]:
# load all images in a directory for good resized
loaded_images_resized_good = list()
for filename in listdir('maskImages/goodResized'):
    # load image
    img_data = image.imread('maskImages/goodResized/' + filename)
    
    # store loaded image
    loaded_images_resized_good.append([img_data,"good"])
    #print('> loaded %s %s' % (filename, img_data.shape))

In [ ]:
# load all images in a directory for bad resized
loaded_images_resized_bad = list()
for filename in listdir('maskImages/badResized'):
    # load image
    img_data = image.imread('maskImages/badResized/' + filename)
    # store loaded image
    loaded_images_resized_bad.append([img_data,"bad"])
   # print('> loaded %s %s' % (filename, img_data.shape))

In [8]:
# load all images in a directory for good grayscale
loaded_images_gray_good = list()
for filename in listdir('maskImages/goodGrayscale'):
    # load image
    img_data = image.imread('maskImages/goodGrayscale/' + filename)
    # store loaded image
    loaded_images_gray_good.append([img_data,"good"])
    #print('> loaded %s %s' % (filename, img_data.shape))

In [18]:
# load all images in a directory for bad grayscale
loaded_images_gray_bad = list()
for filename in listdir('maskImages/badGrayscale'):
    # load image
    img_data = image.imread('maskImages/badGrayscale/' + filename)
    # store loaded image
    loaded_images_gray_bad.append([img_data,"bad"])
   # print('> loaded %s %s' % (filename, img_data.shape))

In [ ]:
#Adding labels to the images as "good" for proper masked images and "bad" otherwise.
#Creating dataframe from the images and appending to form one dataframe for all images.
bad_im_df = pd.DataFrame(loaded_images_resized_bad)
bad_im_df.columns = ["Mask_Data","Label"]
good_im_df = pd.DataFrame(loaded_images_resized_good)
good_im_df.columns = ["Mask_Data","Label"]
df = good_im_df.append(bad_im_df,ignore_index=True)
df.columns = ['Mask_Data','Label']

In [ ]:
#Verifying the count of good and bad images
df['Label'].value_counts()

In [19]:
#Grayscaled images dataframe creation
#Adding labels to the images as "good" for proper masked images and "bad" otherwise.
#Creating dataframe from the images and appending to form one dataframe for all images.
bad_im_df_gray = pd.DataFrame(loaded_images_gray_bad)
bad_im_df_gray.columns = ["Mask_Data","Label"]
good_im_df_gray = pd.DataFrame(loaded_images_gray_good)
good_im_df_gray.columns = ["Mask_Data","Label"]
df_gray = good_im_df_gray.append(bad_im_df_gray,ignore_index=True)
df_gray.columns = ['Mask_Data','Label']

Splitting the data using Stratified shuffle split in 80:20 ratio, test set being 20%.
Keeping random state 42 to make the results reproducible.

In [ ]:
split = StratifiedShuffleSplit(n_splits=6,test_size = 0.2,random_state = 42)
for train_idx,test_idx in split.split(df,df["Label"]):
    train_var = df.iloc[train_idx]
    test_var = df.iloc[test_idx]

In [20]:
#For grayscale images
for train_idx,test_idx in split.split(df_gray,df_gray["Label"]):
    train_var_g = df_gray.iloc[train_idx]
    test_var_g = df_gray.iloc[test_idx]

Creating Validation set with 20% validation data using stratified split.

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train_var["Mask_Data"],train_var["Label"],test_size=0.2,random_state=42)
print(X_train.shape,X_val.shape,y_train.shape,y_val.shape)

In [22]:
#For grayscale images
X_train_g,X_val_g,y_train_g,y_val_g = train_test_split(train_var_g["Mask_Data"],train_var_g["Label"],test_size=0.2,random_state=42)
print(X_train_g.shape,X_val_g.shape,y_train_g.shape,y_val_g.shape)

(1260,) (316,) (1260,) (316,)


Converting Numerical labels into Categorical variables.

In [ ]:
y_train_cat=[0]*len(y_train)
for i in range (0,len(y_train)):
    if y_train.iloc[i] == "good":
        y_train_cat[i]=[1];
    elif y_train.iloc[i] == "bad":
        y_train_cat[i]=[0];

In [23]:
#For Grayscale images
y_train_cat_g=[0]*len(y_train_g)
for i in range (0,len(y_train_g)):
    if y_train_g.iloc[i] == "good":
        y_train_cat_g[i]=[1];
    elif y_train_g.iloc[i] == "bad":
        y_train_cat_g[i]=[0];

In [ ]:
y_val_cat=[0]*len(y_val)
for i in range (0,len(y_val)):
    if y_val.iloc[i] == "good":
        y_val_cat[i]=[1];
    elif y_val.iloc[i] == "bad":
        y_val_cat[i]=[0]

In [25]:
#Grayscale images
y_val_cat_g=[0]*len(y_val_g)
for i in range (0,len(y_val_g)):
    if y_val_g.iloc[i] == "good":
        y_val_cat_g[i]=[1];
    elif y_val_g.iloc[i] == "bad":
        y_val_cat_g[i]=[0]

Converting y train into numpy arrays

In [ ]:
y_train_ar=np.array(y_train_cat)
y_val_ar=np.array(y_val_cat)

In [26]:
# For Grayscale images
y_train_ar_g=np.array(y_train_cat_g)
y_val_ar_g=np.array(y_val_cat_g)

In [28]:
#Having a look at the image to see if it is decent enough for the model to be trained.
plt.imshow(df['Mask_Data'][0])
plt.show()

Converting the images to 4-D for applying CNN. Using reshape to do so.

In [ ]:
X_train_lst =[0] * len(X_train) 
for i in range(0,len(X_train)):
    X_train_lst[i] = np.reshape((X_train.iloc[i]),(200,300,3))

X_train_ar = np.asarray(X_train_lst)
X_train_ar = X_train_ar.astype('float32')
X_train_ar /= 255

X_val_lst=[0] * len(X_val)
for i in range(0,len(X_val)):
    X_val_lst[i] = np.reshape((X_val.iloc[i]),(200,300,3))
    
X_val_ar = np.asarray(X_val_lst)
X_val_ar = X_val_ar.astype('float32')
X_val_ar /= 255


In [30]:
#For Grayscale images
X_train_lst_g =[0] * len(X_train_g) 
for i in range(0,len(X_train_g)):
    X_train_lst_g[i] = np.reshape((X_train_g.iloc[i]),(1024,1024,1))

X_train_ar_g = np.asarray(X_train_lst_g)
X_train_ar_g = X_train_ar_g.astype('float32')
X_train_ar_g /= 255

X_val_lst_g=[0] * len(X_val_g)
for i in range(0,len(X_val_g)):
    X_val_lst_g[i] = np.reshape((X_val_g.iloc[i]),(1024,1024,1))
    
X_val_ar_g = np.asarray(X_val_lst_g)
X_val_ar_g = X_val_ar_g.astype('float32')
X_val_ar_g /= 255


In [31]:
X_val_ar_g.shape

(316, 1024, 1024, 1)

Creating and Pre-processing test data by converting it to categorical and then numpy array.

In [ ]:
X_test_data=test_var["Mask_Data"]
Y_test_data=test_var["Label"]

In [32]:

X_test_data_g=test_var_g["Mask_Data"]
Y_test_data_g=test_var_g["Label"]

Converting target test set to Categorical Variable and making them numpy arrays

In [ ]:
Y_test_cat=[0]*len(Y_test_data)
for i in range (0,len(Y_test_data)):
    if Y_test_data.iloc[i] == 'good':
        Y_test_cat[i]=[1];
    elif Y_test_data.iloc[i] == 'bad':
        Y_test_cat[i]=[0];

Y_test_ar=np.array(Y_test_cat)

In [33]:
#For grayscale images
Y_test_cat_g=[0]*len(Y_test_data_g)
for i in range (0,len(Y_test_data_g)):
    if Y_test_data_g.iloc[i] == 'good':
        Y_test_cat_g[i]=[1];
    elif Y_test_data_g.iloc[i] == 'bad':
        Y_test_cat_g[i]=[0];

Y_test_ar_g=np.array(Y_test_cat_g)

Converting X test data to 4-D and numpy array.

In [ ]:
X_test_set_cat =[0] * len(X_test_data) 
for i in range(0,len(X_test_data)):
    X_test_set_cat[i] = np.reshape((X_test_data.iloc[i]),(200,300,3))

X_test_ar = np.asarray(X_test_set_cat)

X_test_ar = X_test_ar.astype('float32')
X_test_ar /= 255

In [34]:
#For grayscale images
X_test_set_cat_g =[0] * len(X_test_data_g) 
for i in range(0,len(X_test_data_g)):
    X_test_set_cat_g[i] = np.reshape((X_test_data_g.iloc[i]),(1024,1024,1))

X_test_ar_g = np.asarray(X_test_set_cat_g)

X_test_ar_g = X_test_ar_g.astype('float32')
X_test_ar_g /= 255

In [ ]:
#Building CNN model with Maxpooling, Dropout and "relu" as the activation function.

In [35]:
#For grayscale images
#For grayscale images
model_g = Sequential()
model_g.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train_ar_g.shape[1:]))
model_g.add(GaussianNoise(0.1))
model_g.add(Activation('relu'))
model_g.add(Conv2D(32, (3, 3)))
model_g.add(Activation('relu'))
model_g.add(MaxPooling2D(pool_size=(2, 2)))
model_g.add(Dropout(0.25))

model_g.add(Conv2D(64, (3, 3), padding='same'))
model_g.add(GaussianNoise(0.1))
model_g.add(Activation('relu'))
model_g.add(Conv2D(64, (3, 3)))
model_g.add(Activation('relu'))
model_g.add(MaxPooling2D(pool_size=(2, 2)))
model_g.add(Dropout(0.25))

model_g.add(Flatten())
model_g.add(Dense(50))
model_g.add(Activation('relu'))
model_g.add(Dropout(0.5))
model_g.add(Dense(1, activation='sigmoid'))

In [36]:
#Compiling the model with loss function as "binary_crossentropy", since we have binary classification.
model_g.compile(loss='binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy',patience=5, verbose=1, mode='max') #Mode="max" to take weights for maximum accuracy

In [37]:
#fitting model
Model_g=model_g.fit(X_train_ar_g, y_train_ar_g,
              batch_size=4,
              epochs=5,
              validation_data=(X_val_ar_g, y_val_ar_g),
              shuffle=True,callbacks=[es])

Epoch 1/5
315/315 [==============================] - 4197s 13s/step - loss: 2.8046 - accuracy: 0.5127 - val_loss: 0.6937 - val_accuracy: 0.4335
Epoch 2/5
315/315 [==============================] - 4150s 13s/step - loss: 0.6932 - accuracy: 0.5016 - val_loss: 0.6940 - val_accuracy: 0.4335
Epoch 3/5
315/315 [==============================] - 4192s 13s/step - loss: 0.6932 - accuracy: 0.5056 - val_loss: 0.6945 - val_accuracy: 0.4335
Epoch 4/5
315/315 [==============================] - 4221s 13s/step - loss: 0.6932 - accuracy: 0.4913 - val_loss: 0.6949 - val_accuracy: 0.4335
Epoch 5/5
315/315 [==============================] - 4139s 13s/step - loss: 0.6932 - accuracy: 0.5071 - val_loss: 0.6953 - val_accuracy: 0.4335


The validation accuracy of the model is ~43%.

In [38]:
model_g.save(r'C:\Users\praga\Downloads\ChromDownloads\maskNomaskModelGray.h5') #Saving the grascale model

In [3]:
#Model summary
model_load_g = load_model(r"C:\Users\praga\Downloads\ChromDownloads\maskNomaskModelGray.h5")
model_load_g.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1024, 1024, 32)    320       
_________________________________________________________________
gaussian_noise (GaussianNois (None, 1024, 1024, 32)    0         
_________________________________________________________________
activation (Activation)      (None, 1024, 1024, 32)    0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1022, 1022, 32)    9248      
_________________________________________________________________
activation_1 (Activation)    (None, 1022, 1022, 32)    0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 511, 511, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 511, 511, 32)      0

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train_ar.shape[1:]))
model.add(GaussianNoise(0.1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(GaussianNoise(0.1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(2))
model.add(Dense(1, activation='sigmoid'))
#model.add(Activation('softmax'))

In [ ]:
model.compile(loss='binary_crossentropy',metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy',patience=5, verbose=1, mode='max')

In [ ]:
#fitting model
Model1=model.fit(X_train_ar, y_train_ar,
              batch_size=4,
              epochs=5,
              validation_data=(X_val_ar, y_val_ar),
              shuffle=True,callbacks=[es])

In [ ]:
model.save(r'C:\Users\praga\Downloads\ChromDownloads\maskNomaskModel.h5')

In [ ]:
model_load = load_model(r"C:\Users\praga\OneDrive\Desktop\mask_recog_ver2.h5")
model_load.summary()

Plotting the learning curve.

In [ ]:
plt.plot(Model1.history['accuracy'], color='red')
plt.plot(Model1.history['val_accuracy'],color='green')
plt.title('Accuracy plot of training data')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train_set', 'Validation_set'], loc='upper left')
plt.show()

plt.plot(Model1.history['loss'], color='red')
plt.plot(Model1.history['val_loss'], color='green')
plt.title('Loss plot on training data')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train_set', 'Validation_set'], loc='upper right')
plt.show()

Plotting the Loss of model on training data

Evaluating the model on test set 

In [ ]:
scores = model.evaluate(X_test_ar, Y_test_ar, verbose=True)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Predicting using the model

In [ ]:
y_pred=model.predict_classes(X_test_ar)

Checking the confusion matrix to see TP, TN, FP and FN.

In [ ]:
confusion_matrix(Y_test_ar, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix

labels = [0, 1]
cm = confusion_matrix(Y_test_ar, y_pred, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
#from sklearn import ConfusionMatrixDisplay

cm = confusion_matrix(Y_test_ar, y_pred, 11='all')
cmd = sklearn.metrics.ConfusionMatrixDisplay(cm, display_labels=['mask','noMask'])
cmd.plot()

Checking Precision and recall scores for the model

In [ ]:
print('Precision score:%.2f'%precision_score(Y_test_ar, y_pred))
print('Recall score:%.2f'%recall_score(Y_test_ar, y_pred))

Trying diff model by adding layers

In [ ]:
model1 = Sequential()

model1.add(Conv2D(16, (3, 3), padding='same', input_shape=X_train_ar.shape[1:]))
model1.add(GaussianNoise(0.2))
model1.add(Activation("relu"))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train_ar.shape[1:]))
model1.add(GaussianNoise(0.2))
model1.add(Activation('relu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.5))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(GaussianNoise(0.1))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))


model1.add(Flatten())
model1.add(Dense(50))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
#model1.add(Dense(2))
model1.add(Dense(1, activation='sigmoid'))
#model1.add(Activation('softmax'))

In [ ]:
model1.compile(loss='binary_crossentropy',metrics=['accuracy'])
early_stop = EarlyStopping(monitor='val_accuracy',patience=2, verbose=1, mode='max',restore_best_weights=True)

In [ ]:
#fitting model
Model2=model1.fit(X_train_ar, y_train_ar,
              batch_size=4,
              epochs=5,
              validation_data=(X_val_ar, y_val_ar),
              shuffle=True,callbacks=[early_stop])

In [ ]:
scores = model1.evaluate(X_test_ar, Y_test_ar, verbose=True)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
y_pred1=model1.predict_classes(X_test_ar)

In [ ]:
confusion_matrix(Y_test_ar, y_pred1)

In [ ]:
#Precision and Recall Scores
print('Precision score:%.2f'%precision_score(Y_test_ar, y_pred1))
print('Recall score:%.2f'%recall_score(Y_test_ar, y_pred1))

In [ ]:
#Plotting the confusion matrix
from sklearn.metrics import confusion_matrix

labels = ['mask', 'noMask']
cm = confusion_matrix(Y_test_ar, y_pred1, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()